<a href="https://colab.research.google.com/github/drfperez/utilities/blob/main/PDF2Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:


# =====================================================
# 🧮 OCR MATEMÀTIC — DOCX + PDF SEARCHABLE (GOOGLE COLAB)
# =====================================================

dpi = 350
IDIOMES = "cat+spa+eng"
crear_pdf_searchable = True  # ✅ generar PDF amb OCR

# ---------- INSTAL·LACIÓ ----------
!apt-get update -qq
!apt-get install -y -qq \
    poppler-utils tesseract-ocr \
    tesseract-ocr-cat tesseract-ocr-spa tesseract-ocr-eng \
    ghostscript pngquant

!pip install -q pytesseract pdf2image python-docx ocrmypdf Pillow numpy opencv-python-headless

# ---------- IMPORTS ----------
import os, re, cv2, numpy as np
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import pytesseract, ocrmypdf
from docx import Document

TESS_CONFIG = "--oem 1 --psm 6 -c preserve_interword_spaces=1"

# ---------- FUNCIONS ----------
def deskew(img):
    g = cv2.bitwise_not(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    coords = np.column_stack(np.where(g > 0))
    if len(coords) < 30:
        return img
    angle = cv2.minAreaRect(coords)[-1]
    angle = -(90 + angle) if angle < -45 else -angle
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, M, (w, h),
                          flags=cv2.INTER_CUBIC,
                          borderMode=cv2.BORDER_REPLICATE)

def preprocess(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.fastNlMeansDenoising(g, None, 10, 7, 21)
    clahe = cv2.createCLAHE(2.5, (8,8))
    g = clahe.apply(g)
    t = cv2.adaptiveThreshold(
        g, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 31, 9
    )
    return cv2.cvtColor(t, cv2.COLOR_GRAY2RGB)

def clean_math_text(t):
    t = t.replace("\r", "\n")
    t = re.sub(r'-(\n)', '', t)
    t = re.sub(r'[ \t]+', ' ', t)
    t = re.sub(r'\n{3,}', '\n\n', t)
    t = re.sub(r'[\x00-\x1f\x7f]', '', t)

    fixes = {"<=":"≤", ">=":"≥", "!=":"≠",
             " - ":" − ", " x ":" × ", " / ":" ÷ ",
             " O ":" 0 ", " l ":" 1 "}
    for k, v in fixes.items():
        t = t.replace(k, v)
    return t.strip()

# ---------- PUJAR PDF ----------
uploaded = files.upload()
if not uploaded:
    raise SystemExit("❌ Cap PDF seleccionat")

pdf_path = next(iter(uploaded))
base = os.path.splitext(pdf_path)[0]

# ---------- OCR → DOCX ----------
images = convert_from_path(pdf_path, dpi=dpi)
doc = Document()

for i, img in enumerate(images):
    print(f"🧮 OCR pàgina {i+1}/{len(images)}")
    cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv = preprocess(deskew(cv))

    text = pytesseract.image_to_string(
        Image.fromarray(cv),
        lang=IDIOMES,
        config=TESS_CONFIG
    )

    doc.add_heading(f"Pàgina {i+1}", level=2)
    doc.add_paragraph(clean_math_text(text))
    if i < len(images)-1:
        doc.add_page_break()

# ---------- DESAR DOCX ----------
docx_out = f"OCR_MATES_{base}.docx"
doc.save(docx_out)
files.download(docx_out)  # ✅ descarrega DOCX

# ---------- GENERAR PDF SEARCHABLE AMB OCR ----------
if crear_pdf_searchable:
    pdf_out = f"OCR_MATES_searchable_{base}.pdf"
    ocrmypdf.ocr(
        pdf_path,
        pdf_out,
        language=IDIOMES,
        deskew=True,
        optimize=3,
        skip_text=True,         # 🔑 només OCR a pàgines sense text
        oversample=300,
        png_quality=85,         # 🔑 evita errors Colab
        tagged_pdf_mode="ignore"
    )
    files.download(pdf_out)     # ✅ descarrega PDF

print("✅ OCR matemàtic complet — DOCX + PDF")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Saving digitalproblemes.pdf to digitalproblemes (11).pdf
🧮 OCR pàgina 1/7
🧮 OCR pàgina 2/7
🧮 OCR pàgina 3/7
🧮 OCR pàgina 4/7
🧮 OCR pàgina 5/7
🧮 OCR pàgina 6/7
🧮 OCR pàgina 7/7


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The program 'jbig2' could not be executed or was not found on your
system PATH.  This program is recommended when using the --optimize {2,3} arguments,
but not required, so we will proceed.  For best results, install the program.



...

Output()

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ OCR matemàtic complet — DOCX + PDF


In [14]:

# =====================================================
# 🧮 OCR MATEMÀTIC — DOCX + PDF SEARCHABLE (FINAL)
# =====================================================

dpi = 350
IDIOMES = "cat+spa+eng"
crear_pdf_searchable = True  # ✅ generar PDF amb OCR

# ---------- INSTAL·LACIÓ ----------
!apt-get update -qq
!apt-get install -y -qq \
    poppler-utils tesseract-ocr \
    tesseract-ocr-cat tesseract-ocr-spa tesseract-ocr-eng \
    ghostscript pngquant > /dev/null

!pip install -q pytesseract pdf2image python-docx ocrmypdf Pillow numpy opencv-python-headless > /dev/null

# ---------- IMPORTS ----------
import os, re, cv2, numpy as np
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import pytesseract, ocrmypdf
from docx import Document

TESS_CONFIG = "--oem 1 --psm 6 -c preserve_interword_spaces=1"

# ---------- FUNCIONS ----------
def deskew(img):
    g = cv2.bitwise_not(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    coords = np.column_stack(np.where(g > 0))
    if len(coords) < 30:
        return img
    angle = cv2.minAreaRect(coords)[-1]
    angle = -(90 + angle) if angle < -45 else -angle
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, M, (w, h),
                          flags=cv2.INTER_CUBIC,
                          borderMode=cv2.BORDER_REPLICATE)

def preprocess(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.fastNlMeansDenoising(g, None, 10, 7, 21)
    clahe = cv2.createCLAHE(2.5, (8,8))
    g = clahe.apply(g)
    t = cv2.adaptiveThreshold(
        g, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 31, 9
    )
    return cv2.cvtColor(t, cv2.COLOR_GRAY2RGB)

def clean_math_text(t):
    t = t.replace("\r", "\n")
    t = re.sub(r'-(\n)', '', t)
    t = re.sub(r'[ \t]+', ' ', t)
    t = re.sub(r'\n{3,}', '\n\n', t)
    t = re.sub(r'[\x00-\x1f\x7f]', '', t)

    fixes = {"<=":"≤", ">=":"≥", "!=":"≠",
             " - ":" − ", " x ":" × ", " / ":" ÷ ",
             " O ":" 0 ", " l ":" 1 "}
    for k, v in fixes.items():
        t = t.replace(k, v)
    return t.strip()

# ---------- PUJAR PDF ----------
uploaded = files.upload()
if not uploaded:
    raise SystemExit("❌ Cap PDF seleccionat")

pdf_path = next(iter(uploaded))
base = os.path.splitext(pdf_path)[0]

# ---------- OCR → DOCX ----------
images = convert_from_path(pdf_path, dpi=dpi)
doc = Document()

for i, img in enumerate(images):
    cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv = preprocess(deskew(cv))

    text = pytesseract.image_to_string(
        Image.fromarray(cv),
        lang=IDIOMES,
        config=TESS_CONFIG
    )

    doc.add_heading(f"Pàgina {i+1}", level=2)
    doc.add_paragraph(clean_math_text(text))
    if i < len(images)-1:
        doc.add_page_break()

# ---------- DESAR DOCX ----------
docx_out = f"OCR_MATES_{base}.docx"
doc.save(docx_out)
files.download(docx_out)

# ---------- GENERAR PDF SEARCHABLE AMB OCR ----------
if crear_pdf_searchable:
    pdf_out = f"OCR_MATES_searchable_{base}.pdf"
    # ⚡ només UN paràmetre per evitar errors
    ocrmypdf.ocr(
        pdf_path,
        pdf_out,
        language=IDIOMES,
        deskew=True,
        optimize=1,          # optimització segura sense jbig2
        force_ocr=True,      # ✅ força OCR a totes les pàgines
        tagged_pdf_mode="ignore",
        png_quality=85       # evita errors Colab
    )
    files.download(pdf_out)

# ---------- MISSATGE FINAL NET ----------
print("✅ OCR matemàtic complet — DOCX + PDF")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Saving digitalproblemes.pdf to digitalproblemes (10).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Output()

...

Output()

Possible reasons for this include:
--deskew was issued, causing transcoding.
--force-ocr (or --mode force) was issued, causing transcoding.
The optional dependency 'jbig2' was not found, so some image optimizations could not be attempted.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ OCR matemàtic complet — DOCX + PDF
